In [ ]:
cd ..

In [ ]:
import copy
import os

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [ ]:
import echofilter.plotting
import echofilter.raw

In [ ]:
def declare_cc_cmap(cmap, name):
    """
    Register a colorcet colormap in matplotlib.pyplot.

    Parameters
    ----------
    cmap : list of hex str
        List of colors in the colormap, with equispaced
        samples. Each entry should be a hexadecimal
        string.
    name : str
        Name of the colormap. The colormap will be
        available in matplotlib with this name, and can
        be used with `plt.set_cmap(name)`.
    """
    n = len(cmap)
    R = [int(h.lstrip("#")[:2], 16) / 255 for h in cmap]
    G = [int(h.lstrip("#")[2:4], 16) / 255 for h in cmap]
    B = [int(h.lstrip("#")[4:], 16) / 255 for h in cmap]

    R = [(i / (n - 1), v, v) for i, v in enumerate(R)]
    G = [(i / (n - 1), v, v) for i, v in enumerate(G)]
    B = [(i / (n - 1), v, v) for i, v in enumerate(B)]

    cdict = {"red": R, "green": G, "blue": B}

    plt.register_cmap(name=name, data=cdict, lut=n)

In [ ]:
try:
    import colorcet as cc

    declare_cc_cmap(cc.fire, "fire")
    print("Declared fire colormap.")
    declare_cc_cmap(cc.rainbow, "rainbow")
    print("Declared rainbow colormap.")
except ImportError:
    print("colorcet not installed")

In [ ]:
turbulence_color2 = echofilter.plotting.TURBULENCE_COLOR_DARK
bottom_color2 = echofilter.plotting.BOTTOM_COLOR_DARK

In [ ]:
root_data_dir = "/media/scott/scratch/Datasets/dsforce/surveyExports"

# first val sample for stationary
dataset = "MinasPassage"
sample = "december2017/december2017_D20180108-T045216_D20180108-T102216"

# first val sample for mobile
# dataset = 'mobile'
# sample = 'Survey05/Survey05_GR1_S1W_survey5'

# sample = 'Survey07/Survey07_GR4_N5W_survey7'

In [ ]:
transect = echofilter.raw.manipulate.load_decomposed_transect_mask(
    os.path.join(root_data_dir, dataset, sample),
)

In [ ]:
plt.figure(figsize=(15, 9))
echofilter.plotting.plot_transect(transect, x_scale="index")
plt.show()

In [ ]:
fname_top1 = os.path.join(root_data_dir, dataset, sample + "_turbulence.evl")
fname_top2 = os.path.join(root_data_dir, dataset, sample + "_air.evl")
fname_bot = os.path.join(root_data_dir, dataset, sample + "_bottom.evl")

if os.path.isfile(fname_top1):
    t_top, d_top = echofilter.raw.loader.evl_loader(fname_top1)
elif os.path.isfile(fname_top2):
    t_top, d_top = echofilter.raw.loader.evl_loader(fname_top2)
else:
    t_top = d_top = None
if os.path.isfile(fname_bot):
    t_bot, d_bot = echofilter.raw.loader.evl_loader(fname_bot)
else:
    t_bot = d_bot = None

In [ ]:
processed_dir = "/home/scott/Documents/git/deepsense/dsforce/echofilter/processed/"
fname_top = os.path.join(processed_dir, dataset, sample + "_Sv_raw.csv.top.evl")
fname_bot = os.path.join(processed_dir, dataset, sample + "_Sv_raw.csv.bottom.evl")

t_top_gen, d_top_gen = echofilter.raw.loader.evl_loader(fname_top)
t_bot_gen, d_bot_gen = echofilter.raw.loader.evl_loader(fname_bot)

In [ ]:
plt.plot(
    np.arange(transect["timestamps"].shape[0]),
    np.interp(transect["timestamps"], t_top_gen, d_top_gen),
    turbulence_color2,
)
plt.plot(
    np.arange(transect["timestamps"].shape[0]),
    np.interp(transect["timestamps"], t_bot_gen, d_bot_gen),
    bottom_color2,
)
plt.gca().invert_yaxis()

In [ ]:
plt.figure(figsize=(15, 9))
echofilter.plotting.plot_transect(
    transect, x_scale="time" if dataset == "mobile" else "index"
)

if dataset == "mobile":
    tt = transect["timestamps"] - transect["timestamps"][0]
else:
    tt = np.arange(transect["timestamps"].shape[0])
plt.plot(
    tt,
    np.interp(transect["timestamps"], t_top_gen, d_top_gen),
    turbulence_color2,
    linewidth=2,
)
plt.plot(
    tt,
    np.interp(transect["timestamps"], t_bot_gen, d_bot_gen),
    bottom_color2,
    linewidth=2,
)
if dataset == "mobile":
    plt.ylim([0, 67])
    plt.gca().invert_yaxis()
plt.show()

In [ ]:
cmap = "viridis"

In [ ]:
plt.figure(figsize=(15, 9))
echofilter.plotting.plot_transect(
    transect,
    x_scale="time" if dataset == "mobile" else "index",
    turbulence_color="k",
    bottom_color="k",
    surface_color="k",
    passive_color="k",
    removed_color="k",
    cmap=cmap,
)

if dataset == "mobile":
    tt = transect["timestamps"] - transect["timestamps"][0]
else:
    tt = np.arange(transect["timestamps"].shape[0])
plt.plot(
    tt,
    np.interp(transect["timestamps"], t_top_gen, d_top_gen),
    "w",
    linewidth=2,
)
plt.plot(
    tt,
    np.interp(transect["timestamps"], t_bot_gen, d_bot_gen),
    "w",
    linewidth=2,
)
if dataset == "mobile":
    plt.ylim([0, 67])
else:
    plt.ylim([5, 50])
    plt.xlim([0, 1000])

plt.gca().invert_yaxis()
plt.show()